In [1]:
import pandas as pd
import numpy as np

In [2]:
def loadDataset():
    dfTrain=pd.read_csv('Data/train.csv')
    dfDev=pd.read_csv('Data/dev.csv')
    X_train=dfTrain['padded_indices'].apply(eval).tolist()
    X_train=np.array(X_train, dtype=np.int32)
    Y_train = np.array(dfTrain['label'], dtype=np.int32).reshape(-1,1)

    X_dev=dfDev['padded_indices'].apply(eval).tolist()
    X_dev=np.array(X_dev, dtype=np.int32)
    Y_dev = np.array(dfDev['label'], dtype=np.int32).reshape(-1,1)

    return X_train,Y_train,X_dev,Y_dev

In [3]:
from joblib import load
emb_matrix=load('Embeddings/Embedding Matrix.joblib')

In [4]:
X_train,Y_train,X_dev,Y_dev=loadDataset()

In [5]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

/home/tensor/anaconda3/envs/tensor_check/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
import tensorflow as tf
from ray import train, tune
# from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.train import RunConfig
from ray.air.integrations.keras import ReportCheckpointCallback
from ray.air.integrations.mlflow import MLflowLoggerCallback

2025-02-16 13:39:53.809735: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 13:39:53.817902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-16 13:39:53.829140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-16 13:39:53.829177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 13:39:53.836890: I tensorflow/core/platform/cpu_feature_gua

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [8]:
import ray
emb_matrix_id = ray.put(emb_matrix)
X_train_id = ray.put(X_train)
Y_train_id = ray.put(Y_train)
X_dev_id= ray.put(X_dev)
Y_dev_id= ray.put(Y_dev)

2025-02-16 13:39:57,513	INFO worker.py:1841 -- Started a local Ray instance.


In [9]:
def train_model(config):
    emb_matrix = ray.get(emb_matrix_id)
    X_train = ray.get(X_train_id)
    Y_train = ray.get(Y_train_id)
    X_dev= ray.get(X_dev_id)
    Y_dev=ray.get(Y_dev_id)
    
    sentence_indices = Input(shape=(510,),dtype='int32')

    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(emb_matrix.shape[0],emb_matrix.shape[1],trainable=False,mask_zero=True)
   
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])

    X = embedding_layer(sentence_indices)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    for i in range(config['LSTM layers']):
        X = LSTM(config[f'units{i+1}'],return_sequences=True)(X)
        # Add dropout with a probability of 0.5
        #X = Dropout(rate = config[f'Dropout rate{i+1}'] )(X) 
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(config['last unit'],return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    #X = Dropout(rate = config['last dropout rate'] )(X) 
    # Propagate X through a Dense layer with 5 units
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation('sigmoid')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=X)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config['learning_rate']),
        loss='binary_crossentropy',
        metrics=['accuracy','binary_crossentropy']
    )

    model.fit(
            X_train, Y_train,
            epochs=50,
            validation_data=(X_dev, Y_dev),
            verbose=0,
            callbacks=[ReportCheckpointCallback(metrics={"accuracy": "accuracy","loss":"binary_crossentropy"})]
        )
    

In [10]:
# Define the search space for hyperparameters
search_space = {
    "last unit": tune.qrandint(64, 512,64),
    "LSTM layers": tune.randint(0, 4),
    "learning_rate": tune.loguniform(1e-6, 1e-1),
    #"last dropout rate":  tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
}
for i in range(0, 4):  # Add units for layers dynamically based on max num_layers
    search_space[f'units{i+1}'] = tune.qrandint(64, 512,64)
    #search_space[f'Dropout rate{i+1}']=tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])


bohb_hyperband = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=50,
    reduction_factor=2,
    metric="loss",
    mode="min",
)

bohb_search = TuneBOHB(
        # space=config_space,  # If you want to set the space manually
        metric="loss",
        mode="min",
    )

bohb_search = tune.search.ConcurrencyLimiter(bohb_search, max_concurrent=4)

# Configure the Tuner and resources
tuner = tune.Tuner(
    tune.with_resources(train_model, {"cpu": 10, "gpu": 1}),
    param_space=search_space,
    tune_config=tune.TuneConfig(
        num_samples=100,
        scheduler=bohb_hyperband,
        search_alg=bohb_search,
        # metric="loss",
        # mode="min",
    ),
    run_config=train.RunConfig(
            name="BOBH LSTM trail 1",
            callbacks=[
                MLflowLoggerCallback(
                    tracking_uri="http://127.0.0.1:5000",
                    experiment_name="LSTM_model_experiment5"  # Databricks MLflow experiment name
                )
            ]
        )
)

# Start tuning and get results
analysis = tuner.fit()


(raylet) /home/tensor/anaconda3/envs/tensor_check/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(pid=147533) 2025-02-16 13:41:15.135844: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=147533) 2025-02-16 13:41:15.145600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=147533) 2025-02-16 13:41:15.159361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registere

RuntimeError: Should not access `trial_driver_staging_path` without setting `trial_dir_name`

In [11]:
print("Best hyperparameters found were: ", analysis.get_best_result(metric="loss", mode="min").config)

NameError: name 'analysis' is not defined

In [16]:
import ray
print(ray.available_resources())  # Check available CPUs and GPUs
print(ray.cluster_resources())   # Check total resources


{'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'bundle_group_353aa18ee4202de3bad23e8bcab701000000': 1000.0, 'bundle_group_0_353aa18ee4202de3bad23e8bcab701000000': 1000.0, 'GPU_group_353aa18ee4202de3bad23e8bcab701000000': 1.0, 'CPU_group_0_353aa18ee4202de3bad23e8bcab701000000': 2.0, 'node:172.21.168.162': 1.0, 'CPU_group_353aa18ee4202de3bad23e8bcab701000000': 2.0, 'CPU': 10.0, 'object_store_memory': 1478542348.0, 'GPU_group_0_353aa18ee4202de3bad23e8bcab701000000': 1.0, 'memory': 3417580340.0}
{'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU_group_0_353aa18ee4202de3bad23e8bcab701000000': 2.0, 'bundle_group_0_353aa18ee4202de3bad23e8bcab701000000': 1000.0, 'GPU_group_353aa18ee4202de3bad23e8bcab701000000': 1.0, 'node:172.21.168.162': 1.0, 'bundle_group_353aa18ee4202de3bad23e8bcab701000000': 1000.0, 'CPU_group_353aa18ee4202de3bad23e8bcab701000000': 2.0, 'CPU': 12.0, 'memory': 3417580340.0, 'GPU_group_0_353aa18ee4202de3bad23e8bcab701000000': 1.0, 'object_store_memor

In [17]:
ray.shutdown()  # Stop Ray
ray.init(ignore_reinit_error=True)  # Restart Ray

2025-02-16 06:54:47,066	INFO worker.py:1841 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.42.1


In [19]:
import gc

gc.collect()


6629